# 1. Library

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import math
import pickle
import matplotlib.pyplot as plt
from scipy.stats import entropy
from hmmlearn.hmm import GaussianHMM
from hmmlearn import hmm
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Masking
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, LSTM, RepeatVector, Masking

# 2. Prep

### 각 제스처 폴더에서 CSV 파일 로딩
### max_seq_len: 가장 긴 시퀀스를 기준으로 패딩할 길이
### pad_sequences: keras의 유틸을 이용해 시퀀스를 동일 길이로 맞춰줌
### → LSTM 오토인코더에 넣기 위해 필수 전처리

In [2]:
# 한글 폰트 설정 
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False 

In [3]:
# ===============================
# 1. 데이터 로딩 
# ===============================
def load_all_gesture_data(data_dir, gesture_names):
    all_sequences = []
    all_labels = []
    
    for gesture in gesture_names:
        gesture_dir = os.path.join(data_dir, gesture)
        if not os.path.isdir(gesture_dir):
            print(f"Directory not found: {gesture_dir}")
            continue
        
        file_pattern = os.path.join(gesture_dir, "*.csv")
        files = glob.glob(file_pattern)
        print(f"Loading {len(files)} files for gesture: '{gesture}'")
        
        for file in files:
            df = pd.read_csv(file)
            seq = df.values  
            all_sequences.append(seq)
            all_labels.append(gesture)
    
    return all_sequences, all_labels

# 기본 디렉토리 및 제스처 폴더 이름 리스트
data_dir = './data'
gesture_names = ['1', '2', '3', '4', 
                 '5', '6', '7',
                 '8', '9', '10']

# 데이터 로딩 =
sequences, labels = load_all_gesture_data(data_dir, gesture_names)
print(f"Total loaded sequences: {len(sequences)}")

# 모든 시퀀스의 최대 길이 계산 및 패딩
max_seq_len = max(seq.shape[0] for seq in sequences)
print("Max sequence length:", max_seq_len)
padded_sequences = pad_sequences(sequences, maxlen=max_seq_len, dtype='float32', 
                                 padding='post', truncating='post')
print(f"Data Shape: {padded_sequences.shape}")

X_train = padded_sequences
y_train = labels
print("Using all sequences for training:", len(X_train))

Loading 100 files for gesture: '1'
Loading 100 files for gesture: '2'
Loading 100 files for gesture: '3'
Loading 100 files for gesture: '4'
Loading 100 files for gesture: '5'
Loading 100 files for gesture: '6'
Loading 100 files for gesture: '7'
Loading 100 files for gesture: '8'
Loading 100 files for gesture: '9'
Loading 100 files for gesture: '10'
Total loaded sequences: 1000
Max sequence length: 82
Data Shape: (1000, 82, 126)
Using all sequences for training: 1000


# 3. Autoencoder
### LSTM 기반 Autoencoder를 사용해 시퀀스를 압축(latent representation)하고 다시 복원
### 이때 Encoder 출력은 HMM 학습의 입력으로 사용.
### 구조: Input → Masking → LSTM(encoder) → LSTM(decoder)
### 목적: latent feature 추출

In [4]:
# ===============================
# 2. LSTM Autoencoder 구축 및 학습
# ===============================
latent_dim = 126
num_features = padded_sequences.shape[2]
input_shape = (max_seq_len, num_features)
inputs = Input(shape=input_shape, name="input_sequence")
masked = Masking(mask_value=0.0, name="masking")(inputs)
encoded = LSTM(latent_dim, return_sequences=True, name="encoder_lstm")(masked)
decoded = LSTM(num_features, return_sequences=True, name="decoder_lstm")(encoded)
autoencoder = Model(inputs, decoded, name="lstm_autoencoder")
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

history = autoencoder.fit(
    X_train, X_train,
    epochs=50,
    batch_size=16,
    validation_split=0.0  
)

# 인코더 모델 추출
encoder_model = Model(inputs, encoded, name="encoder_model")

Model: "lstm_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_sequence (InputLayer)   │ (None, 82, 126)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal (NotEqual)          │ (None, 82, 126)           │               0 │ input_sequence[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ masking (Masking)             │ (None, 82, 126)           │               0 │ input_sequence[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ any (Any)                     │ (None, 82)                │               0 │ not_equal[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ encoder_lstm (LSTM)           │ (None, 82, 126)           │         127,512 │ masking[0][0], any[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoder_lstm (LSTM)           │ (None, 82, 126)           │         127,512 │ encoder_lstm[0][0],        │
│                               │                           │                 │ any[0][0]                  │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 255,024 (996.19 KB)

 Trainable params: 255,024 (996.19 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - loss: 0.0506
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0077
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0031
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0017
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0012
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0010
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 8.2455e-04
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 6.5224e-04
Epoch 9/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 5.5697e-04
Epoch 10/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 4.8000e-04
Epoch 11/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 4.5231e-04
Epoch 12/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 4.0192e-04
Epoch 13/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 3.7891e-04
Epoch 14/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 3.5781e-04
Epoch 15/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 

In [5]:
# # 재구성 결과 확인 
# reconstructed = autoencoder.predict(X_val)
# print("재구성 결과 :")
# print(reconstructed[0])  # 첫 번째 샘플의 재구성 결과 출력

# # 인코더를 이용해 latent feature 추출
# # encoder_model 출력 :: (n_samples, max_seq_len, latent_dim) 
# latent_features = encoder_model.predict(X_val)

# # 마지막 시간 스텝의 hidden state를 사용하여 한 벡터로 표현
# latent_features_last = latent_features[:, -1, :]

# # 각 latent dimension(특징)별 통계값 계산
# mean_values = np.mean(latent_features_last, axis=0)
# min_values = np.min(latent_features_last, axis=0)
# max_values = np.max(latent_features_last, axis=0)
# range_values = max_values - min_values

# # 결과 출력
# print("\n각 latent dimension의 통계값:")
# print("평균 (Mean):")
# print(mean_values)
# print("최소값 (Min):")
# print(min_values)
# print("최대값 (Max):")
# print(max_values)
# print("범위 (Range, Max-Min):")
# print(range_values)

In [6]:
# # 검증 데이터에 대한 autoencoder의 재구성 결과 예측
# reconstructions = autoencoder.predict(X_val)

# latent_features = encoder_model.predict(X_val)
# latent_last = latent_features[:, -1, :]


# unique_gestures = np.unique(y_val)

# for gesture in unique_gestures:
#     # 현재 제스처에 해당하는 인덱스 추출
#     idx = np.where(np.array(y_val) == gesture)[0]
    
#     # 해당 제스처에 해당하는 데이터, 재구성 결과, latent vector 추출
#     gesture_X_val = X_val[idx]
#     gesture_recon = reconstructions[idx]
#     gesture_latent = latent_last[idx]
    
#     print(f"Gesture: {gesture}")
    
#     # 첫 번째 샘플의 autoencoder 재구성 결과 출력
#     print("재구성 결과 (첫 번째 샘플):")
#     print(gesture_recon[0])
    
#     # latent feature에 대한 scalar 통계값 계산
#     mean_values = np.mean(gesture_latent, axis=0)
#     min_values = np.min(gesture_latent, axis=0)
#     max_values = np.max(gesture_latent, axis=0)
#     range_values = max_values - min_values
    
#     print("latent feature 통계값:")
#     print("평균:", mean_values)
#     print("최소값:", min_values)
#     print("최대값:", max_values)
#     print("범위 (Max - Min):", range_values)
#     print("="*50)

# 4. HMM
### 각 제스처에 대해:
### - Autoencoder의 Encoder 출력값을 latent sequence로 얻음
### - Gaussian HMM을 다양한 상태 수로 학습한 뒤, BIC를 기준으로 최적 상태 수를 선택
### → BIC: 모델의 복잡도 + 적합도 고려하는 모델 선택 지표
### 학습된 HMM은 gesture_hmms[gesture_name]에 저장

In [5]:
# ===============================
# 3. HMM 학습 (각 제스처별)
# ===============================
def compute_bic(model, X, lengths):
    logL = model.score(X, lengths)
    n_components = model.n_components
    n_features = X.shape[1]
    n_samples = sum(lengths)
    n_params = (n_components - 1) + n_components*(n_components - 1) + 2 * n_components * n_features
    bic = -2 * logL + n_params * np.log(n_samples)
    return bic

def train_best_hmm_for_gesture(latent_features, lengths, state_range=(10, 16)):
    best_model = None
    best_bic = np.inf
    best_n_states = None
    for n_states in range(state_range[0], state_range[1]):
        model = GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=100, random_state=42)
        try:
            model.fit(latent_features, lengths)
            bic = compute_bic(model, latent_features, lengths)
            if bic < best_bic:
                best_bic = bic
                best_model = model
                best_n_states = n_states
        except Exception as e:
            print(f"n_states={n_states}에서 학습 실패: {e}")
    return best_model, best_n_states, best_bic

gesture_hmms = {}
for gesture in gesture_names:
    indices = [i for i, lab in enumerate(y_train) if lab == gesture]
    if len(indices) == 0:
        print(f"학습 샘플이 부족합니다: {gesture}")
        continue
    
    features_list = []
    lengths = []
    for i in indices:
        seq = X_train[i]  # (seq_length, num_features)
        latent_seq = encoder_model.predict(seq[np.newaxis, ...])[0]
        features_list.append(latent_seq)
        lengths.append(latent_seq.shape[0])
        
    concatenated_features = np.concatenate(features_list, axis=0)
    best_model, best_n_states, best_bic = train_best_hmm_for_gesture(concatenated_features, lengths, state_range=(10, 12)) #16
    gesture_hmms[gesture] = best_model
    print(f"Gesture '{gesture}': 최적 상태 수 = {best_n_states}, BIC = {best_bic}")

print("모든 제스처 HMM 학습 완료.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━

C:\Users\james\anaconda3\envs\my_new_env\lib\site-packages\hmmlearn\hmm.py:352: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Model is not converging.  Current: -inf is not greater than 3077514.8495401056. Delta is -inf


n_states=11에서 학습 실패: startprob_ must sum to 1 (got nan)
Gesture '1': 최적 상태 수 = 10, BIC = -6072698.928129217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━

Model is not converging.  Current: 2866332.8512803563 is not greater than 2866333.3061355324. Delta is -0.4548551761545241
Model is not converging.  Current: 2885710.2496534297 is not greater than 2885713.583683645. Delta is -3.334030215162784


Gesture '2': 최적 상태 수 = 11, BIC = -5745351.479364535
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/ste

...

Gesture '7': 최적 상태 수 = 11, BIC = -6393554.696869627
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/ste

Model is not converging.  Current: 3270764.562441072 is not greater than 3270812.8922689306. Delta is -48.329827858600765
Model is not converging.  Current: 3307546.7044949243 is not greater than 3307553.4089761362. Delta is -6.70448121195659


Gesture '8': 최적 상태 수 = 11, BIC = -6589065.808311145
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/ste

Model is not converging.  Current: 2760829.8314209953 is not greater than 2760830.5712255435. Delta is -0.7398045482113957
Model is not converging.  Current: 2805705.014640511 is not greater than 2805705.210826775. Delta is -0.19618626404553652


Gesture '10': 최적 상태 수 = 11, BIC = -5585345.254671799
모든 제스처 HMM 학습 완료.


# 5. Threshold Model(ergodic model) & 제스처 적출 네트워크
### 여러 제스처 모델을 통합한 Ergodic HMM을 만듭니다.
### 모든 제스처 상태(mean, cov, self-transition)를 합쳐서 하나의 HMM 구성
###  시작(ST), 종료(FT) 상태도 추가해서 자연스러운 구간 탐지가 가능하도록 함
### 이어서 개별 제스처 상태 + 임계치 모델 상태를 결합한 '최종 제스처 적출 네트워크'를 구성
### → 이 모델을 통해 연속된 시퀀스에서 여러 제스처를 탐지 가능

In [6]:
# ===============================
# 4. 임계치 모델 및 제스처 적출 네트워크
# ===============================
def build_ergodic_threshold_model_with_null(gesture_hmms):
    means_list = []
    covars_list = []
    self_trans_list = []
    state_labels = []
    
    for gesture, model in gesture_hmms.items():
        n_states = model.means_.shape[0]
        means_list.append(model.means_)
        covars = model.covars_
        if covars.ndim == 3:
            diag_covars = np.array([np.diag(cov) for cov in covars])
            covars = diag_covars
        covars_list.append(covars)
        self_trans_list.append(np.diag(model.transmat_))
        state_labels.extend([gesture] * n_states)
    
    all_means = np.vstack(means_list)
    all_covars = np.vstack(covars_list)
    all_self_trans = np.concatenate(self_trans_list)
    N = all_means.shape[0]
    n_features = all_means.shape[1]
    
    st_mean = np.zeros((1, n_features))
    st_covar = np.ones((1, n_features))
    st_self = np.array([1.0])
    ft_mean = np.zeros((1, n_features))
    ft_covar = np.ones((1, n_features))
    ft_self = np.array([1.0])
    
    means = np.vstack([st_mean, all_means, ft_mean])
    covars = np.vstack([st_covar, all_covars, ft_covar])
    self_trans = np.concatenate([st_self, all_self_trans, ft_self])
    total_states = N + 2
    
    # transmat = np.zeros((total_states, total_states))
    # transmat[0, 1:total_states-1] = 1.0 / N
    # transmat[-1, -1] = 1.0
    # for i in range(1, total_states-1):
    #     a_ii = self_trans[i]
    #     num_targets = total_states - 2
    #     transmat[i, 1:total_states] = (1 - a_ii) / num_targets
    #     transmat[i, i] = a_ii

    transmat = np.zeros((total_states, total_states))
    # 시작 상태: 첫 번째 상태로의 전이 확률
    transmat[0, 1:total_states-1] = 1.0 / N
    # 종료 상태: 자기 전이만 허용
    transmat[-1, -1] = 1.0

    for i in range(1, total_states-1):
        a_ii = self_trans[i]
        # 시작 상태와 종료 상태를 제외한 상태들로 균등 분배
        num_targets = total_states - 2
        transmat[i, 1:total_states-1] = (1 - a_ii) / num_targets
        transmat[i, i] = a_ii

    ergodic_model = GaussianHMM(n_components=total_states, covariance_type="diag", init_params="")
    startprob = np.zeros(total_states)
    startprob[0] = 1.0
    ergodic_model.startprob_ = startprob
    ergodic_model.transmat_ = transmat
    ergodic_model.means_ = means
    ergodic_model.covars_ = covars
    ergodic_model.n_features = n_features
    new_state_labels = ["ST"] + state_labels + ["FT"]
    
    return ergodic_model, new_state_labels

# def build_ergodic_threshold_model_with_null(gesture_hmms):
#     means_list = []
#     covars_list = []
#     self_trans_list = []
#     state_labels = []
    
#     for gesture, model in gesture_hmms.items():
#         n_states = model.means_.shape[0]
#         means_list.append(model.means_)
#         covars = model.covars_
#         if covars.ndim == 3:
#             diag_covars = np.array([np.diag(cov) for cov in covars])
#             covars = diag_covars
#         covars_list.append(covars)
#         self_trans_list.append(np.diag(model.transmat_))
#         # 수정: ergodic 상태 레이블에 'ergodic_' 접두사 추가
#         state_labels.extend([f"ergodic_{gesture}"] * n_states)  # 변경된 부분
    
#     all_means = np.vstack(means_list)
#     all_covars = np.vstack(covars_list)
#     all_self_trans = np.concatenate(self_trans_list)
#     N = all_means.shape[0]
#     n_features = all_means.shape[1]
    
#     st_mean = np.zeros((1, n_features))
#     st_covar = np.ones((1, n_features))
#     st_self = np.array([1.0])
#     ft_mean = np.zeros((1, n_features))
#     ft_covar = np.ones((1, n_features))
#     ft_self = np.array([1.0])
    
#     means = np.vstack([st_mean, all_means, ft_mean])
#     covars = np.vstack([st_covar, all_covars, ft_covar])
#     self_trans = np.concatenate([st_self, all_self_trans, ft_self])
#     total_states = N + 2
    
#     transmat = np.zeros((total_states, total_states))
#     transmat[0, 1:total_states-1] = 1.0 / N
#     transmat[-1, -1] = 1.0
#     for i in range(1, total_states-1):
#         a_ii = self_trans[i]
#         num_targets = total_states - 2  # 올바른 계산 (N)
#         transmat[i, 1:total_states] = (1 - a_ii) / num_targets
#         transmat[i, i] = a_ii
#     ergodic_model = GaussianHMM(n_components=total_states, covariance_type="diag", init_params="")
#     startprob = np.zeros(total_states)
#     startprob[0] = 1.0
#     ergodic_model.startprob_ = startprob
#     ergodic_model.transmat_ = transmat
#     ergodic_model.means_ = means
#     ergodic_model.covars_ = covars
#     ergodic_model.n_features = n_features
#     new_state_labels = ["ST"] + state_labels + ["FT"]  # ergodic 상태 레이블이 접두사 포함
    
#     return ergodic_model, new_state_labels

def extract_individual_states(gesture_hmms):
    means_list = []
    covars_list = []
    self_trans_list = []
    labels_list = []
    
    for gesture, model in gesture_hmms.items():
        n_states = model.means_.shape[0]
        means_list.append(model.means_)
        covars = model.covars_
        if covars.ndim == 3:
            covars = np.array([np.diag(cov) for cov in covars])
        covars_list.append(covars)
        self_trans_list.append(np.diag(model.transmat_))
        labels_list.extend([gesture] * n_states)
    
    individual_means = np.vstack(means_list)
    individual_covars = np.vstack(covars_list)
    individual_self = np.concatenate(self_trans_list)
    
    return individual_means, individual_covars, individual_self, labels_list

def build_final_model_with_redundant_components(gesture_hmms, ergodic_model, erg_state_labels, eps=1e-10, p_TM=0.1):
    n_features = ergodic_model.n_features
    s_mean = np.zeros((1, n_features))
    s_covar = np.ones((1, n_features))
    s_self = np.array([0.0])
    
    indiv_means, indiv_covars, indiv_self, indiv_labels = extract_individual_states(gesture_hmms)
    erg_means = ergodic_model.means_
    erg_covars = ergodic_model.covars_
    if erg_covars.ndim == 3:
        erg_covars = np.array([np.diag(cov) for cov in erg_covars])
    erg_covars = erg_covars + eps
    erg_self = np.full((erg_means.shape[0],), p_TM)
    
    final_means = np.vstack([s_mean, indiv_means, erg_means])
    final_covars = np.vstack([s_covar, indiv_covars, erg_covars])
    final_self = np.concatenate([s_self, indiv_self, erg_self])
    final_state_labels = ["S"] + indiv_labels + erg_state_labels
    M = final_means.shape[0]
    
    final_transmat = np.zeros((M, M))
    for i in range(M):
        a_i = final_self[i]
        final_transmat[i, i] = a_i
        final_transmat[i, :] += (1 - a_i) / (M - 1)
        final_transmat[i, i] = a_i
    final_startprob = np.zeros(M)
    final_startprob[0] = 1.0
    
    final_model = GaussianHMM(n_components=M, covariance_type="diag", init_params="")
    final_model.n_features = n_features
    final_model.means_ = final_means
    final_model.covars_ = final_covars
    final_model.transmat_ = final_transmat
    final_model.startprob_ = final_startprob
    
    return final_model, final_state_labels



# ergodic 모델 구성
ergodic_model, state_labels = build_ergodic_threshold_model_with_null(gesture_hmms)

# 6. Viterbi 기반 제스처 구간 탐색 및 시각화
### HMM의 Viterbi 알고리즘을 이용해:
### - 시퀀스에서 어떤 구간이 어떤 제스처인지 파악
### - 각 제스처의 시작/끝 프레임을 찾아냄
### 주요 함수:
### - plot_viterbi_scores_by_gesture: 제스처별 로그 확률 점수를 비교하여 시각화
### - detect_gesture_segments: 점수 차이를 기반으로 구간 추정
### - segment_by_viterbi_path: 상태 경로 기반 구간 추출
### - segment_by_viterbi_path_with_candidate_endpoints: 후보 끝점 기반으로 정교한 추출

In [7]:
# ===============================
# 5. Viterbi 점수 플롯 및 끝점 탐색기 구현 (제스처 구간 추출)
# ===============================
def plot_viterbi_scores_by_gesture(continuous_sequence, encoder_model, gesture_hmms, threshold_model, title=""):
    latent_seq = encoder_model.predict(continuous_sequence[np.newaxis, ...])[0]
    T = latent_seq.shape[0]
    frames = np.arange(1, T+1)
    
    model_scores = {}
    for gesture, model in gesture_hmms.items():
        scores = []
        for t in range(1, T+1):
            try:
                logprob, _ = model.decode(latent_seq[:t], algorithm="viterbi")
                scores.append(logprob)
            except Exception as e:
                scores.append(np.nan)
        model_scores[gesture] = scores

    try:
        threshold_logprob, _ = threshold_model.decode(latent_seq, algorithm="viterbi")
    except Exception as e:
        threshold_logprob = np.nan
    threshold_scores = [threshold_logprob] * T
    model_scores["Threshold"] = threshold_scores

    final_threshold = threshold_scores[-1]
    final_gesture = None
    max_diff = -np.inf
    for gesture, scores in model_scores.items():
        if gesture == "Threshold":
            continue
        diff = scores[-1] - final_threshold
        if diff > max_diff:
            max_diff = diff
            final_gesture = gesture
    if max_diff < 0:
        final_gesture = "None"

    plt.figure(figsize=(14, 6))
    colors = plt.cm.tab10.colors
    for i, (model_name, scores) in enumerate(model_scores.items()):
        if model_name == "Threshold":
            plt.plot(frames, scores, marker='s', linestyle='-', color='black', 
                     label="Threshold Model", linewidth=3)
        else:
            plt.plot(frames, scores, marker='o', linestyle='-', 
                     color=colors[i % len(colors)], label=f"Gesture {model_name}")
    plt.xlabel("Frame Index")
    plt.ylabel("Viterbi Score (log probability)")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return final_gesture, model_scores, frames

def detect_gesture_segments(continuous_sequence, encoder_model, gesture_hmms, threshold_model, margin=0.0, min_duration=5):
    latent_seq = encoder_model.predict(continuous_sequence[np.newaxis, ...])[0]
    T = latent_seq.shape[0]
    gesture_segment_info = {}
    
    for gesture, model in gesture_hmms.items():
        gesture_scores = []
        for t in range(1, T+1):
            try:
                logprob, _ = model.decode(latent_seq[:t], algorithm="viterbi")
            except Exception as e:
                logprob = -np.inf
            gesture_scores.append(logprob)
        try:
            threshold_logprob, _ = threshold_model.decode(latent_seq, algorithm="viterbi")
        except Exception as e:
            threshold_logprob = -np.inf
        diff = np.array(gesture_scores) - threshold_logprob
        segments = []
        start = None
        for i, val in enumerate(diff):
            if val > margin and start is None:
                start = i
            elif val <= margin and start is not None:
                end = i - 1
                if (end - start + 1) >= min_duration:
                    segments.append((start, end))
                start = None
        if start is not None:
            end = T - 1
            if (end - start + 1) >= min_duration:
                segments.append((start, end))
        gesture_segment_info[gesture] = {
            "diff": diff,
            "segments": segments,
        }
    return gesture_segment_info

In [11]:
def segment_by_viterbi_path(continuous_sequence, encoder_model, final_model, final_state_labels, min_duration=5):

    latent_seq = encoder_model.predict(continuous_sequence[np.newaxis, ...])[0]
    T = latent_seq.shape[0]
    logprob, state_sequence = final_model.decode(latent_seq, algorithm="viterbi")
    # 각 상태 인덱스를 대응하는 레이블로 변환
    state_labels = [final_state_labels[s] for s in state_sequence]
    
    segments = []
    current_label = None
    start_index = None
    for i, label in enumerate(state_labels):
        # null 상태("S", "ST", "FT", 또는 "None")는 제스처로 취급하지 않음
        if label not in ["S", "ST", "FT", "None"]:
            if current_label is None:
                # 새로운 제스처 구간 시작
                current_label = label
                start_index = i
            elif label != current_label:
                # 제스처가 변경되었으면 이전 구간을 확정
                end_index = i - 1
                if end_index - start_index + 1 >= min_duration:
                    segments.append((start_index, end_index, current_label))
                # 새로운 구간 시작
                current_label = label
                start_index = i
        else:
            # null 상태가 나오면 현재 진행 중인 구간이 있다면 종료 처리
            if current_label is not None:
                end_index = i - 1
                if end_index - start_index + 1 >= min_duration:
                    segments.append((start_index, end_index, current_label))
                current_label = None
                start_index = None
                
    # 끝까지 구간이 진행 중이었다면 마무리
    if current_label is not None:
        end_index = T - 1
        if end_index - start_index + 1 >= min_duration:
            segments.append((start_index, end_index, current_label))
    
    return segments, state_labels

In [12]:
def segment_by_viterbi_path_with_candidate_endpoints(continuous_sequence, encoder_model, final_model, final_state_labels, min_duration=5):

    # latent 시퀀스 생성 및 Viterbi decoding
    latent_seq = encoder_model.predict(continuous_sequence[np.newaxis, ...])[0]
    T = latent_seq.shape[0]
    logprob, state_sequence = final_model.decode(latent_seq, algorithm="viterbi")
    # 각 상태 인덱스를 레이블로 변환
    state_labels = [final_state_labels[s] for s in state_sequence]
    
    # 연속된 제스처 구간(현 제스처) 추출: null 상태 ("S", "ST", "FT", "None")는 제외
    candidate_segments = []
    current_label = None
    current_indices = []
    for i, label in enumerate(state_labels):
        if label not in ["S", "ST", "FT", "None"]:
            # 제스처 상태인 경우
            if current_label is None:
                current_label = label
                current_indices = [i]
            elif label == current_label:
                current_indices.append(i)
            else:
                # 이전 제스처 구간 종료, 새로운 구간 시작
                candidate_segments.append((current_label, current_indices))
                current_label = label
                current_indices = [i]
        else:
            # null 상태가 나오면 현재 진행 중인 제스처 구간 종료
            if current_label is not None:
                candidate_segments.append((current_label, current_indices))
                current_label = None
                current_indices = []
    if current_label is not None and current_indices:
        candidate_segments.append((current_label, current_indices))
    
    # 각 구간에 대해 후보끝점(첫 후보, 마지막 후보) 및 끝점 결정
    final_segments = []
    for idx, (gesture_label, indices) in enumerate(candidate_segments):
        # 현 제스처의 후보끝점: 첫 후보와 마지막 후보
        first_candidate = indices[0]
        last_candidate = indices[-1]
        
        # 기본적으로 끝점 후보는 last_candidate
        endpoint = last_candidate
        
        # 만약 현 제스처 이후에 다른 구간(제스처 구간)이 없다면, endpoint = last_candidate
        if idx == len(candidate_segments) - 1:
            endpoint = last_candidate
        else:
            # 다음 구간(다음 제스처)의 시작점
            next_start = candidate_segments[idx + 1][1][0]
            # 현 제스처 다음 패턴이 제스처인 경우
            # 다음 제스처의 시작점이 현 제스처의 첫 후보보다 앞에 있으면,
            #     현 제스처를 다음 제스처의 일부로 보고 후보끝점을 무시(여기서는 해당 구간을 스킵)
            if next_start < first_candidate:
                # 현재 구간을 무시하고, 다음 구간과 병합(여기서는 출력을 생략)
                continue
            # 다음 제스처의 시작점이 현 제스처의 첫 후보와 마지막 후보 사이에 있으면,
            #     endpoint = last_candidate (즉, 그대로 유지)
            elif first_candidate <= next_start <= last_candidate:
                endpoint = last_candidate
            else:
                # 그 외의 경우에도 endpoint = last_candidate로 결정
                endpoint = last_candidate
        
        # 최소 길이 조건 만족 확인
        if endpoint - first_candidate + 1 >= min_duration:
            final_segments.append((first_candidate, endpoint, gesture_label))
    
    return final_segments, state_labels

In [18]:
def plot_viterbi_scores_by_gesture_v2(continuous_sequence, encoder_model, gesture_hmms, threshold_model, 
                                      title="", scaling_factor=10, dramatic_multiplier=2, 
                                      target_max=-0.1, threshold_target=-0.2, dynamic_segments=None):
    # latent feature 추출
    latent_seq = encoder_model.predict(continuous_sequence[np.newaxis, ...])[0]
    T = latent_seq.shape[0]
    # 누적 로그 확률 차분은 t=1부터 T까지 계산하므로 diff 배열 길이는 T-1, 프레임 번호는 2부터 T까지
    frames = np.arange(2, T+1)

    model_scores = {}
    # 각 제스처 모델에 대해 누적 로그 확률 계산 및 프레임 간 변화량(diff) 계산
    for gesture, model in gesture_hmms.items():
        log_probs = []
        for t in range(1, T+1):
            try:
                logprob, _ = model.decode(latent_seq[:t], algorithm="viterbi")
                log_probs.append(logprob)
            except Exception as e:
                log_probs.append(np.nan)
        # 인접 프레임 간 변화량 계산 후 scaling_factor와 dramatic_multiplier 적용
        delta_log_probs = np.diff(log_probs) * scaling_factor * dramatic_multiplier
        # 각 제스처는 최대값이 target_max가 되도록 shift
        current_max = np.nanmax(delta_log_probs)
        shift = current_max - target_max
        model_scores[gesture] = delta_log_probs - shift

    # 임계치 모델 계산: dramatic_multiplier 적용 후 threshold_target에 맞춰 shift
    threshold_log_probs = []
    for t in range(1, T+1):
         try:
             logprob, _ = threshold_model.decode(latent_seq[:t], algorithm="viterbi")
             threshold_log_probs.append(logprob)
         except Exception as e:
             threshold_log_probs.append(np.nan)
    threshold_delta_log_probs = np.diff(threshold_log_probs) * dramatic_multiplier
    current_threshold_max = np.nanmax(threshold_delta_log_probs)
    threshold_shift = current_threshold_max - threshold_target
    model_scores["Threshold"] = threshold_delta_log_probs - threshold_shift

    # Dynamic Segments가 제공되면, 해당 구간(프레임)만 선택하여 플롯
    if dynamic_segments is not None:
        mask = np.zeros(T-1, dtype=bool)
        for (start, end, label) in dynamic_segments:
            i_start = max(0, start)      
            i_end = min(T-2, end - 1)      
            mask[i_start:i_end+1] = True
        frames = frames[mask]
        for key in model_scores.keys():
            model_scores[key] = model_scores[key][mask]

    # 최종 제스처 결정: 마지막 프레임의 Δ log p 값 기준으로 각 제스처와 Threshold의 차이를 비교
    final_threshold_delta = model_scores["Threshold"][-1]
    final_gesture = None
    max_diff = -np.inf
    for gesture, deltas in model_scores.items():
        if gesture == "Threshold":
            continue
        diff = deltas[-1] - final_threshold_delta
        if diff > max_diff:
            max_diff = diff
            final_gesture = gesture
    if max_diff < 0:
        final_gesture = "None"

    # 결과 플롯 그리기
    plt.figure(figsize=(14, 6))
    colors = plt.cm.tab10.colors
    for i, (model_name, deltas) in enumerate(model_scores.items()):
        if model_name == "Threshold":
            plt.plot(frames, deltas, marker='s', linestyle='-', color='black', 
                     label="Threshold Model", linewidth=3)
        else:
            plt.plot(frames, deltas, marker='o', linestyle='-', 
                     color=colors[i % len(colors)], label=f"Gesture {model_name}")
    plt.xlabel("Frame Index")
    plt.ylabel("Δ Viterbi Score (Δ log probability)")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return final_gesture, model_scores, frames

# 7. Result
### 테스트용 연속 시퀀스를 로드하여 위에서 학습한 모델에 적용
### → Encoder를 통해 latent sequence를 추출한 뒤,
### → 최종 모델로부터 제스처 구간을 탐지
### 출력: (시작 프레임, 끝 프레임, 제스처 라벨) 리스트

In [14]:
# # ===============================
# # 6. 테스트 
# # ===============================

# p_TM = 1.00E-01  # p(TM) 값 

# final_model, final_state_labels = build_final_model_with_redundant_components(
#     gesture_hmms, ergodic_model, state_labels, eps=1e-10, p_TM=p_TM
# )
# print(f"최종 모델 상태 수: {final_model.n_components}")
# gesture_word = "힘겹다"
# test_folder = f"data/test_{gesture_word}"
# csv_files = sorted(glob.glob(os.path.join(test_folder, "*.csv")))

# for csv_file in csv_files:
#     df_test = pd.read_csv(csv_file)
#     observations = df_test.values
#     print(f"\nLoaded continuous test data shape for file {csv_file}: {observations.shape}")
    
#     # 기존 Viterbi 점수 플롯과 최종 제스처 판별
#     plot_title = f"Viterbi Scores for {os.path.basename(csv_file)})"
#     final_gesture, model_scores, frames = plot_viterbi_scores_by_gesture_v2(
#         observations, encoder_model, gesture_hmms, final_model, title=plot_title
#     )
#     print(f"Final Determined Gesture for {os.path.basename(csv_file)}: {final_gesture}")
    
#     # 동적 구간 결정 실행
#     dynamic_segments = segment_by_viterbi_path(
#     observations, encoder_model, final_model, final_state_labels,
#     min_duration=5
#     )

#     print(f"Dynamic Segments: {dynamic_segments}")

In [24]:
p_TM = 1.0 # p(TM) 값 

final_model, final_state_labels = build_final_model_with_redundant_components(
    gesture_hmms, ergodic_model, state_labels, eps=1e-10, p_TM=p_TM #1e-10
)
print(f"최종 모델 상태 수: {final_model.n_components}")

csv_files = glob.glob("data/GSN/*.csv")

for test_csv in csv_files:
    print(f"Processing file: {test_csv}")
    df_test = pd.read_csv(test_csv)
    
    observations = df_test.iloc[:, 2:].values
    print(f"\nLoaded continuous test data shape for file {test_csv}: {observations.shape}")
    
    # plot_title = f"Viterbi Scores for {os.path.basename(test_csv)} (p(TM)={p_TM})"
    # final_gesture, model_scores, frames = plot_viterbi_scores_by_gesture_v2(
    #     observations, encoder_model, gesture_hmms, final_model, title=plot_title
    # )
    # print(f"Final Determined Gesture for {os.path.basename(test_csv)}: {final_gesture}")
    
    # 구간 결정 
    dynamic_segments = segment_by_viterbi_path(
        observations, encoder_model, final_model, final_state_labels,
        min_duration=11
    )

    print(f"추출된 제스처 구간: {dynamic_segments}")

최종 모델 상태 수: 219
Processing file: data/GSN\1023.csv

Loaded continuous test data shape for file data/GSN\1023.csv: (297, 126)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
추출된 제스처 구간: ([(1, 28, '7'), (31, 108, '10'), (109, 194, '2'), (195, 215, '3'), (225, 296, '3')], ['S', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '1', '1', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '